In [10]:
import pandas as pd
import os
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import bokeh
import os
import openai
from openai import OpenAI
import backoff
import numpy as np
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, ColumnDataSource, output_file, save
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.models import HoverTool
from bokeh.palettes import Category20
import json

In [11]:
# Load csv embeddings file

file = 'LOINC_gpt_embedding.pkl'
df = pd.read_pickle(file)


In [7]:
# Load csv embeddings file

# file1 = 'phecode_gpt_embedding.pkl'
# file2 = 'LOINC_subset_gpt_embedding.pkl'

# df = pd.read_pickle(file1)
# df += pd.read_pickle(file2)

In [8]:
df

,LOINC_NUM,COMPONENT,PROPERTY,TIME_ASPCT,SYSTEM,SCALE_TYP,METHOD_TYP,CLASS,VersionLastChanged,CHNG_TYPE,...,COMMON_ORDER_RANK,HL7_ATTACHMENT_STRUCTURE,EXTERNAL_COPYRIGHT_LINK,PanelType,AskAtOrderEntry,AssociatedObservations,VersionFirstReleased,ValidHL7AttachmentRequest,DisplayName,embeddings
0,100000-9,Health informatics pioneer and the father of L...,Hx,Pt,^Patient,Nar,NaN,H&P.HX,2.74,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.74,NaN,NaN,"[0.0064973183907568455, 0.021421216428279877, ..."
1,100001-7,Health informatics pioneer and cofounder of LOINC,Hx,Pt,^Patient,Nar,NaN,H&P.HX,2.74,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.74,NaN,NaN,"[0.006505629513412714, 0.018597103655338287, 0..."
2,100002-5,Specimen care is maintained,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN,"[0.01835009828209877, 0.007035546004772186, 0...."
3,100003-3,Team communication is maintained throughout care,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN,"[-0.004286017268896103, 0.004707486368715763, ..."
4,100004-1,Demonstrates knowledge of the expected psychos...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN,"[-0.0013429929967969656, 0.0002997028932441026..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102460,99994-6,"Fluid, electrolyte, and acid-base balances are...",Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN,"[0.007286888547241688, 0.014724800363183022, 0..."
102461,99995-3,Respiratory status is maintained at or improve...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN,"[-0.0019765296019613743, 0.0036275875754654408..."
102462,99996-1,Cardiovascular status is maintained at or impr...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN,"[-0.018118860200047493, 0.0013924211962148547,..."
102463,99997-9,Demonstrates &or reports adequate pain control,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN,"[-0.0013925875537097454, -0.015031949616968632..."


# Kmeans and TSNE w Bokeh

In [16]:
'''
Bokeh + TSNE for the LOINC codes
'''
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10
from bokeh.transform import factor_cmap

# Assuming 'df' is your DataFrame and has columns 'LOINC_NUM' and 'COMPONENT'

# Preprocess to create tooltip text columns for LOINC_NUM and COMPONENT
df['LOINC_NUM_tooltip'] = df['LOINC_NUM'].apply(lambda x: f"LOINC_NUM: {x}" if pd.notnull(x) else '')
df['COMPONENT_tooltip'] = df['COMPONENT'].apply(lambda x: f"COMPONENT: {x}" if pd.notnull(x) else '')


# Extract the embeddings into a list of lists (or array of arrays)
embeddings = df['embeddings'].apply(pd.Series).values

# Perform KMeans clustering
n_clusters = 10  # Set the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(embeddings)
df['cluster'] = kmeans.labels_

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=0)
tsne_results = tsne.fit_transform(embeddings)
df['tsne-2d-one'] = tsne_results[:,0]
df['tsne-2d-two'] = tsne_results[:,1]


# Convert the cluster labels to strings
df['cluster'] = df['cluster'].astype(str)

# Create a ColumnDataSource with the cluster labels as strings
source = ColumnDataSource(data=dict(
    x=df['tsne-2d-one'],
    y=df['tsne-2d-two'],
    LOINC_NUM_tooltip=df['LOINC_NUM_tooltip'],
    COMPONENT_tooltip=df['COMPONENT_tooltip'],
    cluster=df['cluster']  # Assuming this is already converted to strings as before
))

# Create the figure
p = figure(width=800, height=800, title="t-SNE and KMeans Visualization",
           tools=("pan,wheel_zoom,box_zoom,reset,hover,save"))

# Use factor_cmap with cluster labels as strings
cluster_colors = factor_cmap('cluster', palette=Category10[10], factors=sorted(df['cluster'].unique()))

# Add the scatter plot
p.scatter('x', 'y', source=source, legend_field='cluster', fill_alpha=0.6,
          size=10, color=cluster_colors, line_color=None)


# Add hover tool
hover = HoverTool()
hover.tooltips = [
    ('Cluster', '@cluster'),
    ('LOINC', '@LOINC_NUM_tooltip'),
    ('Component', '@COMPONENT_tooltip')
]

p.add_tools(hover)

# Before you show the plot, specify the filename for the output.
output_file("LOINC_tsne_kmeans_visualization_2.html", title="LOINC t-SNE and KMeans Visualization")

# Show the plot
show(p)




/Users/ellenbowen/opt/anaconda3/envs/ehail/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
